<a href="https://colab.research.google.com/github/jwoonge/ML-projects/blob/master/11/assignment_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional Neural Network for the classification task on MNIST
20141261 송제웅
<hr>
  
## 0. Import library
<hr>

In [1]:
import torch
from torchvision import datasets, transforms
import torch.nn as nn
import matplotlib.pyplot as plt

## 1. Download and Normalize dataset (MNIST)
---


In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,)),
])

data_path = './MNIST'
data_train = datasets.MNIST(root = data_path, train = False, download = True, transform = transform)
data_test = datasets.MNIST(root = data_path, train = True, download = True, transform = transform)

Extracting ./MNIST/MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/MNIST/raw


Extracting ./MNIST/MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/MNIST/raw



Extracting ./MNIST/MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/MNIST/raw


Extracting ./MNIST/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


## 2. Design Model
---

In [4]:
class CNN_module(nn.Module):
    def __init__(self, num_class=10, size_kernel=5):
        super(CNN_module, self).__init__()
        self.num_class = num_class
        self.size_kernel = size_kernel

        #### Feature Layer ####
        self.conv1 = nn.Conv2d(1, 20, kernel_size=self.size_kernel, stride=1, padding=int((size_kernel-1)/2), bias=True)
        self.conv1 = nn.Conv2d(20, 50, kernel_size=self.size_kernel, stride=1, padding=int((size_kernel-1)/2), bias=True)

        self.conv_layer1 = nn.Sequential(self.conv1, nn.MaxPool2d(kernel_size=2), nn.ELU())
        self.conv_layer2 = nn.Sequential(self.conv2, nn.MaxPool2d(kernel_size=2), nn.ELU())

        self.feature = nn.Sequential(self.conv_layer1, self.conv_layer2)
    
        #### Classifier Layer ####
        self.fc1        = nn.Linear(50*7*7, 50, bias=True)
        self.fc2        = nn.Linear(50, num_classes, bias=True)

        self.fc_layer1  = nn.Sequential(self.fc1, nn.ELU(True))
        self.fc_layer2  = nn.Sequential(self.fc2, nn.ELU(True))

        self.classifier = nn.Sequential(self.fc_layer1, self.fc_layer2)
        self._initialize_weight() 

    def _initialize_weight(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight, gain=math.sqrt(2))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight, gain=math.sqrt(2))
                if m.bias is not None:
                    m.bias.data.zero_()
        
    def forward(self, x):
            x = self.feature(x)
            x = x.view(x.size(0), -1)
            x = self.classifier(x)
            return x
